In [9]:
import numpy as np
import math
from scipy.interpolate import interp1d

import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt

from numpy.fft import fft, ifft , rfft, irfft , fftfreq
from numpy import exp, log, log10, cos, sin, pi, cosh, sinh , sqrt
from classy import Class
from scipy.optimize import fsolve
from scipy.special import gamma
from scipy.special import hyp2f1
import sys,os
from time import time
from scipy.integrate import quad
import scipy.integrate as integrate
from scipy import special
from scipy.special import factorial

In [10]:
#Starting CLASS

z_pk = 0.61
common_settings = {# fixed LambdaCDM parameters
                   'A_s':2.089e-9,
                   'n_s':0.9649,
                   'tau_reio':0.052,
                   'omega_b':0.02237,
                   'h':0.6736,
                   'YHe':0.2425,
#                   'N_eff':3.046,
                   'N_ur':2.0328,
                   'N_ncdm':1,
#                   'N_ncdm':0,
                   'm_ncdm':0.06,
                   # other output and precision parameters
#                    'P_k_max_1/Mpc':100.0,
                   'z_pk':z_pk,
                   'output':'mTk,vTk',
                   'lensing':'no',
                   'P_k_max_h/Mpc': 100.}

M = Class()
M.set(common_settings)
#compute linear LCDM only
M.set({ 'non linear':'no',
        'omega_cdm': 0.1193,
        'z_max_pk': 100000
        #'omega_dmeff': 0.,
        #'omega_cdm':1e-15,
        #'omega_dmeff': 0.1193,
        #'npow_dmeff': 0,
        #'sigma_dmeff': 1e-25,
        #'m_dmeff': 1.0,
        #'dmeff_target': 'hydrogen', 
        #'Vrel_dmeff': 0.
      })

M.compute()

M1 = Class()
M1.set(common_settings)
#compute linear dmeff
M1.set({'non linear':'no',
        #'omega_cdm': 0.1193,
        #'omega_dmeff': 0.,
        'omega_cdm':1e-15,
        'omega_dmeff': 0.1193,
        'npow_dmeff': -4,
        'sigma_dmeff': 1e-41,
        'm_dmeff': 1.0,
        'dmeff_target': 'hydrogen', 
        'Vrel_dmeff': 30.,
        'z_max_pk': 100000
      })

M1.compute()

In [11]:
#Extracting background and thermo

bg_LCDM = M.get_background()
th_LCDM = M.get_thermodynamics()

bg_dmeff = M1.get_background()
th_dmeff = M1.get_thermodynamics()

In [12]:
#calculating sigma

#getting masses
m_dmeff = 1. #GeV
kg_to_ev = 5.60958860380445e35
c = 299792458.
m_dm = m_dmeff * 1e9 / (kg_to_ev * (c**2)) # in kg
m_H = 1.673575e-27 # in kg

#getting temperatures
_k_B_ = 1.3806504e-23
zs = bg_dmeff['z']
T_b = np.array([M1.baryon_temperature(z) for z in zs])
T_dmeff = bg_dmeff['T_dmeff']
print(zs)

#getting velocities
vb2 = T_b / m_H * _k_B_
vdmeff2 = T_dmeff / m_dm * _k_B_
vth2 = vb2 + vdmeff2 # thermal velocity squared
vrel = bg_dmeff['Vrel_dmeff']
vrel2 = vrel * vrel / 3
v2 = vth2 + vrel2
vn = v2 ** (-2)

#getting sigma
sigma = 1e-45 * vn * (c**4)
print(sigma)

[1.00000000e+14 9.89308584e+13 9.78731474e+13 ... 2.17307062e-02
 1.08069579e-02 0.00000000e+00]
[2.24428578e-82 2.29305572e-82 2.39379798e-82 ... 1.17568165e-51
 1.20811795e-51 1.24162246e-51]


In [13]:
#calculating number density

#getting mass density
rho_b = bg_dmeff['(.)rho_b']

#All densities are multiplied by (8piG/3)
#Densities are in units [Mpc^-2]

G = 6.6743015e-11
Mpc_to_m = 3.08567758128e22
rho_b = rho_b * 3 * (c**2) / (8*math.pi*G)
rho_b = rho_b / (Mpc_to_m**2)
n_b = rho_b / m_H
print(n_b)

[2.51070269e+41 2.43103169e+41 2.35388885e+41 ... 2.67796332e-01
 2.59298471e-01 2.51070269e-01]


In [14]:
#calculating mean free path

l = 1 / (sigma * n_b)
l = l / Mpc_to_m
print(l)

[5.75143227e+17 5.81358768e+17 5.75143223e+17 ... 1.02933109e+29
 1.03452303e+29 1.03959602e+29]


In [ ]:
#plotting

plt.plot(zs,l)
plt.xlabel('z')
plt.xscale('log')
plt.ylabel('mean free path [Mpc]')
plt.yscale('log')
plt.show()